In [143]:
# step 1: import modules
import pybamm;from pybamm import constants,exp,tanh,sqrt;
import numpy as np
import matplotlib.pyplot as plt;import matplotlib as mpl;
import pandas as pd   ;import numpy as np;import os;import os;
from scipy.io import savemat,loadmat; 
import traceback;import imageio.v2 as imageio
import sys  
str_path_0 = os.path.abspath(os.path.join(pybamm.__path__[0],'..'))
str_path_1 = os.path.abspath(os.path.join(str_path_0,"wip\Rio_Code\Fun_P3"))
sys.path.append(str_path_1) 
from Fun_P3 import (
    PlotDynamics,
    Plot_Loc_Var,
    Plot_Loc_Var_sol,
    Plot_Single_Static,
    may_cause_error,
)
fs=17; 
font = {'family' : 'DejaVu Sans','size'   : fs}
mpl.rc('font', **font)

In [144]:
def electrolyte_conductivity_Nyman2008Exp_wEC(c_e,c_EC, T):
    x = c_EC / c_e
    coff = 1  
    ratio = ( coff/2 + coff/2 *  tanh((x-4.541*0.5)*1.5))
    sigma_e = (
        ratio * (
        0.1 * 0.06248 * (1+298.15-0.05559) * 
        (c_e/1e3) * (1 - 3.084 *sqrt(c_e/1e3) 
        + 1.33 *(1+ 0.03633 *(exp(1000/298.15))*c_e/1e3)   ) 
        / (1+(c_e/1e3)**4*( 0.00795 *exp(1000/298.15))) )
    )
    return sigma_e
def electrolyte_conductivity_Nyman2008Exp_wcEC(c_e,c_EC, T): # with constant c_EC
    x = 4541 / c_e
    coff = 1  
    ratio = ( coff/2 + coff/2 *  tanh((x-4.541*0.5)*1.5))
    sigma_e = (
        ratio * (
        0.1 * 0.06248 * (1+298.15-0.05559) * 
        (c_e/1e3) * (1 - 3.084 *sqrt(c_e/1e3) 
        + 1.33 *(1+ 0.03633 *(exp(1000/298.15))*c_e/1e3)   ) 
        / (1+(c_e/1e3)**4*( 0.00795 *exp(1000/298.15))) )
    )
    return sigma_e
class RioCallback(pybamm.callbacks.Callback):
    def __init__(self, logfile=None):
        self.logfile = logfile
        self.success  = True
        if logfile is None:
            # Use pybamm's logger, which prints to command line
            self.logger = pybamm.logger
        else:
            # Use a custom logger, this will have its own level so set it to the same
            # level as the pybamm logger (users can override this)
            self.logger = pybamm.get_new_logger(__name__, logfile)
            self.logger.setLevel(pybamm.logger.level)
    
    def on_experiment_error(self, logs):
        self.success  = False
    def on_experiment_infeasible(self, logs):
        self.success  = False

In [145]:
# Step 2: model parameters
De = 3e-10;Dec = 5e-10; Dcross  = 3e-11; Xi=-1.4; t_0plus=0.28
D_ec_sei=5e-21; # 
model_0 = pybamm.lithium_ion.DFN(     
    options={
        "SEI":"solvent-diffusion limited",          
        "SEI film resistance":"distributed",          
        "SEI porosity change":"true",     
        "solvent diffusion": "EC wo refill"     } )
ChemistryChen=pybamm.parameter_sets.Chen2020 
ChemistryChen["electrolyte"] = "lipf6_Nyman2008Exp";
Para_0=pybamm.ParameterValues(chemistry=ChemistryChen);
Para_0['EC transference number'] =    Xi
Para_0['Cation transference number'] =     t_0plus
Para_0['EC Lithium ion cross diffusivity [m2.s-1]'] = Dcross
Para_0['Typical EC Lithium ion cross diffusivity [m2.s-1]'] =  Dcross
Para_0['EC diffusivity in electrolyte [m2.s-1]'] =  Dec
Para_0['Ratio of lithium moles to SEI moles'] =  1
Para_0['EC diffusivity in SEI [m2.s-1]'] =  D_ec_sei
Para_0.update({
    'Electrolyte conductivity [S.m-1]':
    electrolyte_conductivity_Nyman2008Exp_wEC}, )
Para_0['Ratio of lithium moles to SEI moles'] =  1
Para_0.update({"Upper voltage cut-off [V]": 4.22})
Para_0.update({"Lower voltage cut-off [V]": 2.48})

c_e = model_0.variables["Electrolyte concentration [mol.m-3]"]
T = model_0.variables["Cell temperature [K]"]
c_EC = model_0.variables["EC concentration [mol.m-3]"]
model_0.variables["c(EC) over c(Li+)"] = c_EC / c_e
model_0.variables["Electrolyte conductivity [S.m-1]"] =(
    Para_0['Electrolyte conductivity [S.m-1]'](c_e,c_EC, T))
model_0.variables["Electrolyte diffusivity [m2.s-1]"] =(
    Para_0['Electrolyte diffusivity [m2.s-1]'](c_e,c_EC, T))

Confirm: using solvent-diffusion limited
using EC wo refill for Li+


In [146]:
BasicPath = 'D:/OneDrive - Imperial College London/SimDataSave/P3R7/'; 
Target  = 'a4_1_base_run_to_fail/'
if not os.path.exists(BasicPath + Target):
    os.mkdir(BasicPath + Target);

In [147]:
# Report two bugs here (may be ): dynamic plot cannot sincerely refelct what is in the 

In [148]:
# Plot a pair of loc dependent varibles - within one step
def Plot_Loc_Var_sol( sol,x_loc_all, key_all, cycle, step,colormap  ): # for initial solution object
    Num_subplot = len(key_all); # must have 2+ keys
    fig, axs = plt.subplots(1,Num_subplot, figsize=(7*Num_subplot,5),tight_layout=True)
    for i in range(0,Num_subplot):
        x_loc=x_loc_all[i]; key=key_all[i];
        LinesNmum = len(sol.cycles[cycle].steps[step][key].entries[0,:] )
        cmap_i = mpl.cm.get_cmap(colormap, LinesNmum) 
        for j in range(0,LinesNmum):
            axs[i].plot(
                sol.cycles[cycle].steps[step][x_loc].entries[:,0], 
                sol.cycles[cycle].steps[step][key].entries[:,j], '-',
                color=cmap_i(j),)
            axs[i].set_title(key ,   fontdict={'family':'DejaVu Sans','size':fs-1})
            #axs[1].set_ylabel("Potential [V]",   fontdict={'family':'DejaVu Sans','size':fs})
            axs[i].set_xlabel(x_loc,   fontdict={'family':'DejaVu Sans','size':fs})
            
            labels = axs[i].get_xticklabels() + axs[i].get_yticklabels(); [label.set_fontname('DejaVu Sans') for label in labels]
            
            axs[i].tick_params(labelcolor='k', labelsize=fs, width=1) ;  del labels;
            axs[i].ticklabel_format( axis='x', style='sci',scilimits=[-0.01,0.01], useOffset=None, useLocale=None, useMathText=None)
            #axs[i].legend(prop={'family':'DejaVu Sans','size':fs-2},loc='best',frameon=False)  
    return 

In [149]:
# Step 3: run set up model_0 and parameters:
V_max = 4.2;        V_min = 2.5; 
exp_text_1 = [ (
    f"Discharge at 1C until {V_min} V",  
    "Rest for 1 hours",  
    f"Charge at 1C until {V_max} V" ) ]
exp_text_CD = [ (f"Discharge at 1C until {V_min} V",   ) ]
exp_text_RE = [ ("Rest for 1 hours", ) ]
exp_text_CC = [ (f"Charge at 1C until {V_max} V" ) ]
# further Steps if we can catch the seconds information:
ExpList     = [
    exp_text_1,exp_text_1,exp_text_1,
    exp_text_CD,exp_text_RE,exp_text_CC];
SaveAsList  = [
    100,       10,  1, 
    1, 1,1];
Succ_Cyc = []; Sol_All = [];    # contains all solutions and corresponding cycles
tot_target_cyc = 200
i = 0;step_switch = 0
while (i < tot_target_cyc):
    print('try to run %d cycles' % SaveAsList[step_switch])
    expe_i    = pybamm.Experiment( 
        ExpList[step_switch] * 
        SaveAsList[step_switch]) 
    # inherit previous resuls or run model from scratch
    try:   # run the model
        if i == 0: # first time or never succeed, run model from scratch:
            rioCall = RioCallback()  # define callback
            sim_0    = pybamm.Simulation(
                model_0,        experiment = expe_i,
                parameter_values = Para_0,
                solver = pybamm.CasadiSolver(),
                #var_pts=var_pts,
                #submesh_types=submesh_types
                ) #mode="safe"
            sol_0    = sim_0.solve(
                calc_esoh=False,
                save_at_cycles = SaveAsList[step_switch],
                callbacks=rioCall)
            
        else: # succeeded before, 
            model_new = model_old.set_initial_conditions_from(
                sol_old, inplace=False)
            rioCall = RioCallback()  # define callback
            sim_new    = pybamm.Simulation(
                model_new,        experiment = expe_i,
                parameter_values = Para_0,
                solver = pybamm.CasadiSolver(),
                #var_pts=var_pts,
                #submesh_types=submesh_types
                ) #mode="safe"
            sol_new    = sim_new.solve(
                calc_esoh=False,
                save_at_cycles = SaveAsList[step_switch],
                callbacks=rioCall)   
        if rioCall.success == False:
            1/0
    except:
        print('Failed and shorten cycles')
        step_switch += 1
        if (step_switch >= len(SaveAsList)):
            print('Exit as no options left')
            break
    else:        
        if i == 0: 
            model_old = model_0; sol_old = sol_0    
        else: 
            model_old = model_new; sol_old = sol_new
            del model_new,sol_new
        Sol_All.append(sol_old);
        Succ_Cyc.append(SaveAsList[step_switch])
        i += SaveAsList[step_switch]
        print('Succeed! Now it is the %dth cycles' % i)  
        if step_switch <= 2:
            step_switch += 0
        elif step_switch > 2 and step_switch < len(SaveAsList)-1:
            step_switch += 1
            print('Succeed a single step and switch to next step normally')
        else:
            print('Finish last single step and Exit as no options left')
            break



try to run 100 cycles
Succeed! Now it is the 100th cycles
try to run 100 cycles


2022-10-27 00:42:20.381 - [WARNING] callbacks.on_experiment_infeasible(233): 

	Experiment is infeasible: 'event: Maximum voltage' was triggered during 'Rest for 1 hours'. The returned solution only contains up to step 2 of cycle 50. 


Failed and shorten cycles
try to run 10 cycles
Succeed! Now it is the 110th cycles
try to run 10 cycles
Succeed! Now it is the 120th cycles
try to run 10 cycles
Succeed! Now it is the 130th cycles
try to run 10 cycles
Succeed! Now it is the 140th cycles
try to run 10 cycles


2022-10-27 00:45:36.007 - [WARNING] callbacks.on_experiment_infeasible(233): 

	Experiment is infeasible: 'event: Maximum voltage' was triggered during 'Rest for 1 hours'. The returned solution only contains up to step 2 of cycle 10. 


Failed and shorten cycles
try to run 1 cycles
Succeed! Now it is the 141th cycles
try to run 1 cycles
Succeed! Now it is the 142th cycles
try to run 1 cycles
Succeed! Now it is the 143th cycles
try to run 1 cycles
Succeed! Now it is the 144th cycles
try to run 1 cycles
Succeed! Now it is the 145th cycles
try to run 1 cycles
Succeed! Now it is the 146th cycles
try to run 1 cycles
Succeed! Now it is the 147th cycles
try to run 1 cycles
Succeed! Now it is the 148th cycles
try to run 1 cycles
Succeed! Now it is the 149th cycles
try to run 1 cycles


2022-10-27 00:46:37.071 - [WARNING] callbacks.on_experiment_infeasible(233): 

	Experiment is infeasible: 'event: Maximum voltage' was triggered during 'Rest for 1 hours'. The returned solution only contains up to step 2 of cycle 1. 


Failed and shorten cycles
try to run 1 cycles
Succeed! Now it is the 150th cycles
Succeed a single step and switch to next step normally
try to run 1 cycles


2022-10-27 00:46:43.576 - [WARNING] callbacks.on_experiment_infeasible(233): 

	Experiment is infeasible: 'event: Maximum voltage' was triggered during 'Rest for 1 hours'. The returned solution only contains up to step 1 of cycle 1. 


Failed and shorten cycles
try to run 1 cycles
Succeed! Now it is the 151th cycles
Finish last single step and Exit as no options left


In [150]:
# Sol_All[-1] contains the 140th to 150th cycles, 
# among which the 150th cycles only contains the first 2 cycles

print("voltage of the final 2 steps (discharge and rest)")
print(Sol_All[-1].cycles[9].steps[0]["Terminal voltage [V]"].entries)
print(Sol_All[-1].cycles[9].steps[1]["Terminal voltage [V]"].entries)
print ("current is")
print(Sol_All[-1].cycles[9].steps[0]["Current [A]"].entries)
print(Sol_All[-1].cycles[9].steps[1]["Current [A]"].entries)
print("Porosity times concentration") 
print(Sol_All[-1].cycles[9].steps[0]["Porosity times concentration"].entries)
print(Sol_All[-1].cycles[9].steps[1]["Porosity times concentration"].entries)


voltage of the final 2 steps (discharge and rest)


IndexError: list index out of range

In [151]:
Sol_All[1].cycles[0].steps

In [ ]:
print(len(Sol_All))
print(Succ_Cyc)
print(Sol_All)

In [ ]:
Sol_All[-1].cycles

In [152]:
output_variables3 = [
    "Terminal voltage [V]",
    "Current [A]",
    "Electrolyte concentration",
    "Minus div Li+ flux",
    "Li+ source term",
    "Minus div Li+ flux by diffusion",
    "Minus div Li+ flux by migration",
    "Minus div Li+ flux by solvent",
    "EC concentration",
    "Minus div EC flux by diffusion",
    "Minus div EC flux by migration",
    "Minus div EC flux by Li+",
]
quick_plot = pybamm.QuickPlot(
    Sol_All[14].cycles[-1],
    output_variables3,
    variable_limits='tight',time_unit='hours',n_rows=3,
    figsize = (16,12)) #     spatial_unit='mm',
quick_plot.dynamic_plot();

interactive(children=(FloatSlider(value=0.0, description='t', max=0.6016389253336181, step=0.00601638925333618…

In [153]:
output_variables3 = [
    "Terminal voltage [V]",
    "Discharge capacity [A.h]",
    #"Negative electrode potential [V]",
    "Negative electrode porosity",
    "Porosity times concentration",
    "Electrolyte potential [V]",
    "Electrolyte current density [A.m-2]",
    'Electrolyte conductivity [S.m-1]',
    "Electrolyte concentration [mol.m-3]",
    #"Sum of positive electrode interfacial current densities",

    #"Li+ source term refill",
]
quick_plot = pybamm.QuickPlot(
    Sol_All[13].cycles[-1],
    output_variables3,
    variable_limits='fixed',time_unit='hours',n_rows=2,
    figsize = (16,9)) #     spatial_unit='mm',
quick_plot.dynamic_plot();

interactive(children=(FloatSlider(value=0.0, description='t', max=2.2037579950660486, step=0.02203757995066048…

In [ ]:
# PLOT for discharge step of the last solution
fs = 21
font = {'family' : 'DejaVu Sans','size'   : fs}
mpl.rc('font', **font)
Plot_Loc_Var_sol(
    Sol_All[-1],
    ["x [m]","x [m]","x [m]","x [m]"], 
    [
        "Electrolyte concentration [mol.m-3]",
        "EC concentration [mol.m-3]",
        "Electrolyte potential [V]",
        "Electrolyte conductivity [S.m-1]",],
    -1,0,'cool')
Plot_Loc_Var_sol(
    Sol_All[-1],
    ["x_n [m]","x_s [m]","x_p [m]"], 
    [
        "Negative electrode porosity times concentration",
        "Separator porosity times concentration",
        "Positive electrode porosity times concentration",],
    -1,0,'cool')


In [ ]:
# PLOT for rest step of the last solution
fs = 21
font = {'family' : 'DejaVu Sans','size'   : fs}
mpl.rc('font', **font)
Plot_Loc_Var_sol(
    Sol_All[-1],
    ["x [m]","x [m]","x [m]","x [m]"], 
    [
        "Electrolyte concentration [mol.m-3]",
        "EC concentration [mol.m-3]",
        "Electrolyte potential [V]",
        "Electrolyte conductivity [S.m-1]",],
    -1,1,'cool')
Plot_Loc_Var_sol(
    Sol_All[-1],
    ["x_n [m]","x_s [m]","x_p [m]"], 
    [
        "Negative electrode porosity times concentration",
        "Separator porosity times concentration",
        "Positive electrode porosity times concentration",],
    -1,1,'cool')
# some wired things happen during rest, making electrolyte concentration very high 

In [ ]:
# analyse the flux contribution for the rest period:
print("c(Li+) and flux")
Plot_Loc_Var_sol(
    Sol_All[-1],
    ["x [m]","x [m]","x [m]",], 
        ["Electrolyte concentration",
        "Minus div Li+ flux",
        "Li+ source term",],
    -1,1,'cool')
print("Li+ flux breakdown")
Plot_Loc_Var_sol(
    Sol_All[-1],
    ["x [m]","x [m]","x [m]",], 
        ["Minus div Li+ flux by diffusion",
        "Minus div Li+ flux by migration",
        "Minus div Li+ flux by solvent",],
    -1,1,'cool')
print("c(EC) and flux")
Plot_Loc_Var_sol(
    Sol_All[-1],
    ["x [m]","x [m]","x [m]",], 
        ["EC concentration",
        "c(EC) over c(Li+)",
        "Minus div EC flux",], 
    -1,1,'cool')
print("EC flux breakdown")
Plot_Loc_Var_sol(
    Sol_All[-1],
    ["x [m]","x [m]","x [m]",], 
        ["Minus div EC flux by diffusion",
        "Minus div EC flux by migration",
        "Minus div EC flux by Li+",], 
    -1,1,'cool')

In [ ]:
# analyse the flux contribution for the discharge period:
print("c(Li+) and flux")
Plot_Loc_Var_sol(
    Sol_All[-1],
    ["x [m]","x [m]","x [m]",], 
        ["Electrolyte concentration",
        "Minus div Li+ flux",
        "Li+ source term",],
    -1,0,'cool')
print("Li+ flux breakdown")
Plot_Loc_Var_sol(
    Sol_All[-1],
    ["x [m]","x [m]","x [m]",], 
        ["Minus div Li+ flux by diffusion",
        "Minus div Li+ flux by migration",
        "Minus div Li+ flux by solvent",],
    -1,0,'cool')
print("c(EC) and flux")
Plot_Loc_Var_sol(
    Sol_All[-1],
    ["x [m]","x [m]","x [m]",], 
        ["EC concentration",
        "c(EC) over c(Li+)",
        "Minus div EC flux",], 
    -1,0,'cool')
print("EC flux breakdown")
Plot_Loc_Var_sol(
    Sol_All[-1],
    ["x [m]","x [m]","x [m]",], 
        ["Minus div EC flux by diffusion",
        "Minus div EC flux by migration",
        "Minus div EC flux by Li+",], 
    -1,0,'cool')

In [ ]:
print(Sol_All[-1].cycles[-1].steps[1]["Electrolyte conductivity [S.m-1]"].entries[:,-1])


In [ ]:
print(Sol_All[-1].cycles[-1].steps[0]["Electrolyte current density"].entries[:,-1])

print(Sol_All[-1].cycles[-1].steps[1]["Electrolyte current density"].entries[:,-1])

#print(Sol_All[-1].cycles[-1].steps[1]["Electrolyte current density"].entries[:,0])

In [ ]:
# step 4: core: a big loop to run model_0 until it fail
# target: change steps; print out the limit;
# todo in the future: catch the wrong message time and use it to fast loction failure point
# sample information o catch: 
# Simulation error: Maximum number of decreased steps occurred at t=106279.54960562677.
#  D_ec_sei=1e-20; is a nice value to catch errors, can run about 30 1C cycles, 

In [ ]:
# step 5: post-proessing: for only 1C discharge, print the last 1C dischagre capacity, 
# then critical variables as static lines or dynamic plots in the last cycle 
# for ageing with RPT, print the last RPT, then discharge capacity in every ageing cycle, 
# then for last cycle, critical variables
# critical variables should include: terminal voltage, electrolyte potential; porosity; 
# electrolyte diffusivity and conductivity; electrode soc,  